In [1]:
import numpy as np
import pickle
from L63_V import L63 as model
from gauss_initialize import gauss_initialize as spin_up
from scipy.integrate import odeint
from bootstrap_unbiased import bootstrap as PF
from generate_observations import gen_obs

In [2]:
## System Parameters

# experiment name
name ='test_run'
name = name + '.p'
save_file = open(name,'wb')

# state dimension
state_dim = 3

# observation dimension
obs_dim = 3

# observation operator
H = np.eye(state_dim)

# Define the number of particles in the ensemble
particle_number = 50

# Spin time
spin_end = 1000

# Time step
dt = .01

# Spin interval
spin = np.linspace(0,spin_end,spin_end/dt)

# Obs Err variance (% of climate variance) 
obs_var = 0.01

# prior covariance
p_cov = np.eye(state_dim)*.5

# Analysis performed after tanl steps
tanl = 10

# Number of Analyses (after the analysis at time zero)
nanl = 100

# Experiment length defined
exp_len = tanl*nanl

# Interval of integration including time zero
exp_int = np.linspace(0,exp_len*dt,exp_len+1)

In [3]:
## generate first prior and observations

# generate trajectory on the attractor for the truth
# and produce the initial prior for this state
[truth,prior] = spin_up(model,state_dim,particle_number,spin,dt,p_cov,obs_var)

# integrate the truth for the length of the model, and return this trajector and the
# noisy observations of the state
[truth_traj,obs,Q] = gen_obs(model,truth,H,exp_int,nanl,tanl,obs_var)

In [4]:
## particle filter step

pdf_series = PF(model,state_dim,prior,particle_number,exp_int,nanl,tanl,obs,Q)

In [5]:
pickle.dump(pdf_series, save_file)
save_file.close()